In [7]:
import sys
import json
import vk_api

LIKES = 1  # Нужно ли записывать людей, которые лайнкули фото
GROUP_URL = 'dubki'  # Короткий url или id группы

def auth_handler():
    """ При двухфакторной аутентификации вызывается эта функция.
    """
    # Код двухфакторной аутентификации
    key = input("Enter authentication code: ")
    # Если: True - сохранить, False - не сохранять.
    remember_device = True
    return key, remember_device

def authorization():
    LOGIN = input("Login: ")
    PASSWORD = input("Password: ")
    vk_session = vk_api.VkApi(
        LOGIN, PASSWORD,
        # функция для обработки двухфакторной аутентификации
        auth_handler=auth_handler
    )
    try:
        vk_session.auth()
    except vk_api.AuthError as error_msg:
        print(error_msg)
        return
    return vk_session.get_api()

def choose_album():
    GROUP_URL = input("Group name or URL: ")
    group_id = vk.groups.getById(group_id=GROUP_URL)[0]['id']
    albums = vk.photos.getAlbums(owner_id=-group_id)
    print('Choose an album:')
    for i in range(albums['count']):
        print('%d. %s' % (i+1, albums['items'][i]['title']))
    return albums['items'][int(input('Input album number: '))]['id'], group_id

def get_photos(album_id, owner_id):
    photos = vk.photos.get(owner_id=-owner_id, album_id=album_id)
    print('Number of photos: %d' % photos['count'])
    if (photos['count'] == 0):
        return 0, 0
    photos_count = input('Number of photos to be loaded: ') 
    photos_ids = []
    photos_src = []
    if int(photos_count) < 1001:
        for i in range(int(photos_count)):
            photos_ids.append(photos['items'][i]['id'])
            photos_src.append(photos['items'][i]['sizes'])
    else:
        offset = 0
        while int(photos_count) > offset:
            for i in range(min(1000, int(photos_count) - offset)):
                photos_ids.append(photos['items'][i]['id'])
                photos_src.append(photos['items'][i]['sizes'])
            offset += 1000
            photos = vk.photos.get(owner_id=-owner_id, album_id=album_id, offset=offset)
    return photos_ids, photos_src

def get_url(owner_id, photo_id):
    return 'https://vk.com/photo%s_%s' % (-owner_id, photo_id)

def get_likes(owner_id, photo_id):
    likes = vk.likes.getList(type='photo', owner_id=-owner_id, item_id=photo_id, count=1000)
    users = []
    if likes['count'] < 1001:
        for i in range(likes['count']):
            users.append('https://vk.com/id' + str(likes['items'][i]))
    else:
        offset = 0
        while likes['count'] > offset:
            for i in range(min(1000, likes['count'] - offset)):
                users.append('https://vk.com/id' + str(likes['items'][i]))
            offset += 1000
            likes = vk.likes.getList(type='photo', owner_id=-owner_id, item_id=photo_id, offset=offset, count=1000)
    return {'count': likes['count'], 'users': users}

def main():
    global vk
    vk = authorization()
    album_id, owner_id = choose_album()
    photos_ids, photos_src = get_photos(album_id, owner_id)
    result = []
    if (photos_ids == 0):
        print("There are no photos in this album")
    else:
        for i in range(len(photos_ids)):
            url = get_url(owner_id, photos_ids[i])
            src = photos_src[i]
            likes = get_likes(owner_id, photos_ids[i]) if LIKES else {}
            result.append({
                'url': url,
                'src': src,
                'likes': likes,
            })
            sys.stdout.write('\rLoading: %i out of %i' % (i + 1, len(photos_ids)))
        file = open('photos.json', 'w', encoding='utf8')
        json.dump(result, file, ensure_ascii=False, indent=4)
        file.close()
        print("\nYou can now see the result in the photos.json file")

if __name__ == '__main__':
    main()

Login: +79875034344
Password: e@rnMM!999
Group name or URL: dubki
Choose an album:
1. ДУБКИ-2
2. чилл дэйс
3. Футбол: 7 vs 9
4. Гитарные вечера 2013-2014
5. GARAGE SALE DUBKI 21.09.2014
6. Меню группы
7. Весна, лето, осень, зима и снова весна
8. Тестирование интернета в общежитии
9. Служебное
10. потерянное
11. Осень в Дубках (13.10.2013)
12. Мемы Дубков
13. Дубковский быт
14. Дорога.
15. Столовая Дубков
16. Тапочки Дубков
17. Мистер общежития 2012
18. Плетение кос
19. Фотоконкурс«СделайМнеСтрашно» или кошмар на улице ДенисаДавыдова
20. Дубайские вороны
21. Виды с балкона общаги
22. Дубки в МЕЛу
23. Новый Год 2011
24. Новогодний фотоконкурс Дубков!
25. Новый аватар для группы: конкурс
26. Основной альбом
Input album number: 2
Number of photos: 47
Number of photos to be loaded: 3
Loading: 3 out of 3
You can now see the result in the photos.json file
